In [1]:
from datetime import datetime
import MetaTrader5 as mt5
#import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [2]:
#connect to mt5
if not mt5.initialize():
   print("Unable to connect to MT5")
   mt5.shutdown()

In [3]:
symbol_info = mt5.symbol_info("EURUSD")
print("EurUsd: ")
print("Spread: ", symbol_info.spread, " Digits: ", symbol_info.digits)
print("Bid: ", symbol_info.bid, " Ask: ", symbol_info.ask)

EurUsd: 
Spread:  3  Digits:  5
Bid:  1.21243  Ask:  1.21246


In [4]:
date = datetime.now().strftime("%Y %m %d %H %M").split()
date[3] = (int(date[3]) + 3) % 24
for num, d in enumerate(date):
   date[num] = int(d)

eurusd_rates = mt5.copy_rates_from("EURUSD", mt5.TIMEFRAME_M5, datetime(date[0],date[1],date[2],date[3],date[4]), 10080)
# for eurusd in eurusd_rates:
#    print(datetime.fromtimestamp(eurusd[0]))
print(type(eurusd_rates))
print(eurusd_rates)

<class 'numpy.ndarray'>
[(1609436100, 1.22535, 1.22544, 1.22438, 1.22462, 519, 0, 0)
 (1609436400, 1.22462, 1.22474, 1.22305, 1.22363, 653, 0, 0)
 (1609436700, 1.22365, 1.22371, 1.22265, 1.22289, 610, 0, 0) ...
 (1613764500, 1.21227, 1.21274, 1.21222, 1.21269, 296, 1, 0)
 (1613764800, 1.21271, 1.21277, 1.21237, 1.2124 , 260, 1, 0)
 (1613765100, 1.21239, 1.21245, 1.21239, 1.21243,  13, 1, 0)]


In [5]:
test = np.array(eurusd_rates, dtype=[('timestamp', 'int64'), ("open", "float64"), ("high", "float64"),
    ("low", "float64"), ("close", "float64"), ("volume", "int64"), ("spread", "int64"), ("real_volume", "int64")])
print("Spread:")
print("moy: ", test["spread"].mean(), " std: ", test["spread"].std(), " min: ", test["spread"].min(), " max: ", test["spread"].max(), " per25: ", np.percentile(test["spread"], 25), " median: ", np.percentile(test["spread"], 50), " per75: ", np.percentile(test["spread"], 75))

Spread:
moy:  0.7989087301587302  std:  3.167502834403709  min:  0  max:  53  per25:  0.0  median:  0.0  per75:  0.0


In [6]:
import talib
import pandas as pd

rsi = talib.RSI(test["close"], timeperiod=14)
print(len(rsi))
print(rsi[:20])

10080
[        nan         nan         nan         nan         nan         nan
         nan         nan         nan         nan         nan         nan
         nan         nan 31.02836879 30.33333333 38.27469012 41.52725428
 36.2126509  40.37620555]


In [7]:
eurusd_M5 = mt5.copy_rates_from_pos("EURUSD", mt5.TIMEFRAME_M5, 0, 10_000) # 99_999
eurusd_M30 = mt5.copy_rates_from_pos("EURUSD", mt5.TIMEFRAME_M30, 0, 10_000) # 99_999

print(eurusd_M5[-1])
print(eurusd_M30[-1])
#mt5.last_error()
pd_eurusd_M5 = pd.DataFrame(eurusd_M5)
pd_eurusd_M30 = pd.DataFrame(eurusd_M30)


cci5 = talib.CCI(pd_eurusd_M5["high"], pd_eurusd_M5["low"], pd_eurusd_M5["close"], timeperiod= 14)

print("CCI  M5: ", cci5.iloc[-1])

(1613765100, 1.21239, 1.21245, 1.21239, 1.21243, 13, 1, 0)
(1613764800, 1.21271, 1.21277, 1.21237, 1.21243, 273, 1, 0)
CCI  M5:  -88.50574712646714


In [8]:
# Bollinger Band
pd_eurusd_M5["upperBBands_M5"], pd_eurusd_M5["middleBBands_M5"], pd_eurusd_M5["lowerBBands_M5"] = talib.BBANDS(pd_eurusd_M5["close"], timeperiod=20, nbdevup=2, nbdevdn=2, matype=0)

# Double Exponential Moving Average
pd_eurusd_M5["DEMA_M5"] = talib.DEMA(pd_eurusd_M5["close"], timeperiod= 21)

# MidPoint over period
pd_eurusd_M5["Midpoint_M5"] = talib.MIDPOINT(pd_eurusd_M5["close"], timeperiod= 14)

# Midpoint Price over period
pd_eurusd_M5["Midprice_M5"] = talib.MIDPRICE(pd_eurusd_M5["high"], pd_eurusd_M5["low"], timeperiod= 14)

# Parabolic SAR
pd_eurusd_M5["SAR_M5"] = talib.SAR(pd_eurusd_M5["high"], pd_eurusd_M5["low"], acceleration=0.02, maximum=0.2)

# Aroon Oscillator
pd_eurusd_M5["Aroon_M5"] = talib.AROONOSC(pd_eurusd_M5["high"], pd_eurusd_M5["low"], timeperiod=14)

# Balance Of Power
pd_eurusd_M5["BOP_M5"] = talib.BOP(pd_eurusd_M5["open"], pd_eurusd_M5["high"], pd_eurusd_M5["low"], pd_eurusd_M5["close"])

# CCI
pd_eurusd_M5["CCI_M5"] = talib.CCI(pd_eurusd_M5["high"], pd_eurusd_M5["low"], pd_eurusd_M5["close"], timeperiod= 14)

# MACD
pd_eurusd_M5["MACD_M5"], pd_eurusd_M5["MACDSignal_M5"], pd_eurusd_M5["MACDHist_M5"] = talib.MACD(pd_eurusd_M5["close"], fastperiod=12, slowperiod=26, signalperiod=9)

# Momentum
pd_eurusd_M5["Mom_M5"] = talib.MOM(pd_eurusd_M5["close"], timeperiod=10)

# Percentage Price Oscillator
pd_eurusd_M5["PPO_M5"] = talib.PPO(pd_eurusd_M5["close"], fastperiod=12, slowperiod=26, matype=0)

# Rate of change
pd_eurusd_M5["ROC_M5"] = talib.ROC(pd_eurusd_M5["close"], timeperiod=10)

# Relative Strength Index
pd_eurusd_M5["RSI_M5"] = talib.RSI(pd_eurusd_M5["close"], timeperiod=14)

# Stochastic
pd_eurusd_M5["Stoc_slowk_M5"], pd_eurusd_M5["Stoc_slowd_M5"] = talib.STOCH(pd_eurusd_M5["high"], pd_eurusd_M5["low"], pd_eurusd_M5["close"], fastk_period=5, slowk_period=3, slowk_matype=0, slowd_period=3, slowd_matype=0)

# TRIX
pd_eurusd_M5["TRIX_M5"] = talib.TRIX(pd_eurusd_M5["close"], timeperiod=30)

# Ultimate Oscillator
pd_eurusd_M5["UltOsc_M5"] = talib.ULTOSC(pd_eurusd_M5["high"], pd_eurusd_M5["low"], pd_eurusd_M5["close"], timeperiod1=7, timeperiod2=14, timeperiod3=28)

# Williams' %R
pd_eurusd_M5["WillamsR_M5"] = talib.WILLR(pd_eurusd_M5["high"], pd_eurusd_M5["low"], pd_eurusd_M5["close"], timeperiod=14)

# Chaikin A/D Oscillator
pd_eurusd_M5["ADOsc_M5"] = talib.ADOSC(pd_eurusd_M5["high"], pd_eurusd_M5["low"], pd_eurusd_M5["close"], pd_eurusd_M5["tick_volume"], fastperiod=3, slowperiod=10)

# On Balance Volume
pd_eurusd_M5["OBV_M5"] = talib.OBV(pd_eurusd_M5["close"], pd_eurusd_M5["tick_volume"])

# True Range
pd_eurusd_M5["TrueRange_M5"] = talib.TRANGE(pd_eurusd_M5["high"], pd_eurusd_M5["low"], pd_eurusd_M5["close"])

In [9]:
#Merge M5 and M30 Dataframes
pd_eurusd = pd_eurusd_M5.merge(pd_eurusd_M30, how="left", left_on= "time", right_on= "time")

# print(pd_eurusd[:40])
# test = pd.DataFrame(pd_eurusd, columns=["time", "high_x", "low_x", "SAR_M5"])
# print(test[:40])
# prev_SAR = 1.098328
# prev_af = 0.04
# EP = 1.09842
# print(prev_SAR + prev_af * (EP - prev_SAR))

In [10]:
# Remove first empty lines of M30

for i in range(0,7):
    if pd_eurusd["time"].iloc[0] % 1800 == 0:
        break
    else:
        pd_eurusd = pd_eurusd.drop([i])

# pd_eurusd[0:12]

In [11]:
# Adjust M30 open, high, low, close values

for i in range(0, len(pd_eurusd)):
    if pd_eurusd["time"].iloc[i] % 1800 == 0:
        pd_eurusd["open_y"].iloc[i] = pd_eurusd["open_x"].iloc[i]
        pd_eurusd["high_y"].iloc[i] = pd_eurusd["high_x"].iloc[i]
        pd_eurusd["low_y"].iloc[i] = pd_eurusd["low_x"].iloc[i]
        pd_eurusd["close_y"].iloc[i] = pd_eurusd["close_x"].iloc[i]
    else:
        pd_eurusd["open_y"].iloc[i] = pd_eurusd["open_y"].iloc[i-1]
        pd_eurusd["close_y"].iloc[i] = pd_eurusd["close_x"].iloc[i]
        if pd_eurusd["high_x"].iloc[i] > pd_eurusd["high_x"].iloc[i-1]:
            pd_eurusd["high_y"].iloc[i] = pd_eurusd["high_x"].iloc[i]
        else:
            pd_eurusd["high_y"].iloc[i] = pd_eurusd["high_y"].iloc[i-1]
        if pd_eurusd["low_x"].iloc[i] < pd_eurusd["low_x"].iloc[i-1]:
            pd_eurusd["low_y"].iloc[i] = pd_eurusd["low_x"].iloc[i]
        else:
            pd_eurusd["low_y"].iloc[i] = pd_eurusd["low_y"].iloc[i-1]


# pd_eurusd[0:10]

In [12]:
# Adjust M30 volume

for i in range(0, len(pd_eurusd)):
    if pd_eurusd["time"].iloc[i] % 1800 == 0:
        pd_eurusd["tick_volume_y"].iloc[i] = pd_eurusd["tick_volume_x"].iloc[i]
    else:
        pd_eurusd["tick_volume_y"].iloc[i] = pd_eurusd["tick_volume_x"].iloc[i] + pd_eurusd["tick_volume_y"].iloc[i-1]

# pd_eurusd[0:15]

In [13]:
# Bollinger bands M30
timeperiod = 20
nbdev = 2

count = 0
midBand = np.array([])
upBand = np.array([])
lowBand = np.array([])
tpTemp = np.array([])
#closeTemp = np.array([])


for i in range(0, len(pd_eurusd)):
    if count < timeperiod-1:
        if pd_eurusd["time"].iloc[i] % 1800 == 1500:
            count += 1
            tp = (pd_eurusd["high_y"].iloc[i] + pd_eurusd["low_y"].iloc[i] + pd_eurusd["close_y"].iloc[i]) / 3
            tpTemp = np.append(tpTemp, tp)
            #closeTemp = np.append(closeTemp, pd_eurusd["close_y"].iloc[i])
            midBand = np.append(midBand, [np.nan])
            upBand = np.append(upBand, [np.nan])
            lowBand = np.append(lowBand, [np.nan])
        else:
            midBand = np.append(midBand, [np.nan])
            upBand = np.append(upBand, [np.nan])
            lowBand = np.append(lowBand, [np.nan])
    else:
        if pd_eurusd["time"].iloc[i] % 1800 == 1500:
            # Calcul midBand, return midBand and new smaTemp
            # Calcul upBand, return upBand
            # Calcul lowBand, return lowBand and new closeTemp
            # Calcul midBand, return midBand
            tp = (pd_eurusd["high_y"].iloc[i] + pd_eurusd["low_y"].iloc[i] + pd_eurusd["close_y"].iloc[i]) / 3
            tpTemp = np.append(tpTemp, tp)
            smaTemp = talib.SMA(tpTemp, timeperiod= timeperiod)
            midBand = np.append(midBand, [smaTemp[-1]])
            # Calcul sigma
            std_tp = np.std(tpTemp)
            # Calcul upBand, return upBand
            temp_up = smaTemp[-1] + nbdev * std_tp
            upBand = np.append(upBand, [temp_up])
            # Calcul lowBand, return lowBand
            temp_low = smaTemp[-1] - nbdev * std_tp
            lowBand = np.append(lowBand, [temp_low])
            # Delete first tpTemp value to stay with tiemperiod-1 values
            tpTemp = np.delete(tpTemp, 0)
        else:
            # Calcul midBand, return midBand
            tp = (pd_eurusd["high_y"].iloc[i] + pd_eurusd["low_y"].iloc[i] + pd_eurusd["close_y"].iloc[i]) / 3
            temp_tp = np.append(tpTemp, tp)
            smaTemp = talib.SMA(temp_tp, timeperiod= timeperiod)
            midBand = np.append(midBand, [smaTemp[-1]])
            # Calcul sigma
            std_tp = np.std(temp_tp)
            # Calcul upBand, return upBand
            temp_up = smaTemp[-1] + nbdev * std_tp
            upBand = np.append(upBand, [temp_up])
            # Calcul lowBand, return lowBand
            temp_low = smaTemp[-1] - nbdev * std_tp
            lowBand = np.append(lowBand, [temp_low])

pd_eurusd["upperBBands_M30"] = upBand
pd_eurusd["middleBBands_M30"] = midBand
pd_eurusd["lowerBBands_M30"] = lowBand

In [14]:
# Double Exponential Moving Average ** On Close ** timeperiod = 21
timeperiod = 21

count = 0
dema = np.array([])
closeTemp = np.array([])
emaTemp = np.array([])

for i in range(0, len(pd_eurusd)):
    if count < timeperiod-1:
        if pd_eurusd["time"].iloc[i] % 1800 == 1500:
            count += 1
            closeTemp = np.append(closeTemp, pd_eurusd["close_y"].iloc[i])
            dema = np.append(dema, np.nan)
        else:
            dema = np.append(dema, np.nan)
    else:
        if count < 2 * timeperiod -1:
            if pd_eurusd["time"].iloc[i] % 1800 == 1500:
                count += 1
                closeTemp = np.append(closeTemp, pd_eurusd["close_y"].iloc[i])
                ema1 = talib.EMA(closeTemp, timeperiod= timeperiod)
                emaTemp = np.append(emaTemp, ema1[-1])
                closeTemp = np.delete(closeTemp, 0)
                dema = np.append(dema, np.nan)
            else:
                dema = np.append(dema, np.nan)
        else:
            if pd_eurusd["time"].iloc[i] % 1800 == 1500:
                closeTemp = np.append(closeTemp, pd_eurusd["close_y"].iloc[i])
                ema1 = talib.EMA(closeTemp, timeperiod= timeperiod)
                emaTemp = np.append(emaTemp, ema1[-1])
                ema2 = talib.EMA(emaTemp, timeperiod= timeperiod)
                calc = 2 * ema1[-1] - ema2[-1]
                dema = np.append(dema, calc)
                closeTemp = np.delete(closeTemp, 0)
                emaTemp = np.delete(emaTemp, 0)
            else:
                c_temp = np.append(closeTemp, pd_eurusd["close_y"].iloc[i])
                ema1 = talib.EMA(c_temp, timeperiod= timeperiod)
                ema_tp = np.append(emaTemp, ema1[-1])
                ema2 = talib.EMA(ema_tp, timeperiod= timeperiod)
                calc = 2 * ema1[-1] - ema2[-1]
                dema = np.append(dema, calc)

pd_eurusd["DEMA_M30"] = dema

In [15]:
# MidPoint over period * on close * timeperiod = 14
timeperiod = 14

count = 0
midpoint = np.array([])
closeTemp = np.array([])

for i in range(0, len(pd_eurusd)):
    if count < timeperiod-1:
        if pd_eurusd["time"].iloc[i] % 1800 == 1500:
            count += 1
            closeTemp = np.append(closeTemp, pd_eurusd["close_y"].iloc[i])
            midpoint = np.append(midpoint, np.nan)
        else:
            midpoint = np.append(midpoint, np.nan)
    else:
        if pd_eurusd["time"].iloc[i] % 1800 == 1500:
            closeTemp = np.append(closeTemp, pd_eurusd["close_y"].iloc[i])
            calc = (closeTemp.max() + closeTemp.min()) / 2
            midpoint = np.append(midpoint, calc)
            closeTemp = np.delete(closeTemp, 0)
        else:
            c_temp = np.append(closeTemp, pd_eurusd["close_y"].iloc[i])
            calc = (c_temp.max() + c_temp.min()) / 2
            midpoint = np.append(midpoint, calc)

pd_eurusd["Midpoint_M30"] = midpoint

In [16]:
# Midpoint Price over period * on high, low * timeperiod = 14
timeperiod = 14

count = 0
midpoint = np.array([])
highTemp = np.array([])
lowTemp = np.array([])

for i in range(0, len(pd_eurusd)):
    if count < timeperiod-1:
        if pd_eurusd["time"].iloc[i] % 1800 == 1500:
            count += 1
            highTemp = np.append(highTemp, pd_eurusd["high_y"].iloc[i])
            lowTemp = np.append(lowTemp, pd_eurusd["low_y"].iloc[i])
            midpoint = np.append(midpoint, np.nan)
        else:
            midpoint = np.append(midpoint, np.nan)
    else:
        if pd_eurusd["time"].iloc[i] % 1800 == 1500:
            highTemp = np.append(highTemp, pd_eurusd["high_y"].iloc[i])
            lowTemp = np.append(lowTemp, pd_eurusd["low_y"].iloc[i])
            calc = (highTemp.max() + lowTemp.min()) / 2
            midpoint = np.append(midpoint, calc)
            highTemp = np.delete(highTemp, 0)
            lowTemp = np.delete(lowTemp, 0)
        else:
            h_temp = np.append(highTemp, pd_eurusd["high_y"].iloc[i])
            l_temp = np.append(lowTemp, pd_eurusd["low_y"].iloc[i])
            calc = (h_temp.max() + l_temp.min()) / 2
            midpoint = np.append(midpoint, calc)

pd_eurusd["Midprice_M30"] = midpoint

In [17]:
# Parabolic SAR * acceleration = 0.02, maximum = 0.2


acc = 0.02
maximum = 0.2

trend = 0 # uptrend => 1, donwtrend => -1
SAR = np.array([])
ext_point = 0
prev_pos = 0

i_start = 0
init_bool = True

# initialisation, get trend

for i in range(0, len(pd_eurusd)):
    if pd_eurusd["time"].iloc[i] % 1800 == 1500:
        if init_bool:
            i_start = i
            SAR = np.append(SAR, np.nan)
            init_bool = False
        else:
            # uptrend
            if pd_eurusd["high_y"].iloc[i] > pd_eurusd["high_y"].iloc[i_start] and pd_eurusd["low_y"].iloc[i] > pd_eurusd["low_y"].iloc[i_start]:
                trend = 1
                SAR = np.append(SAR, pd_eurusd["low_y"].iloc[i_start])
                ext_point = pd_eurusd["low_y"].iloc[i_start]
                i_start = i
                prev_pos = i

                break 
            # donwtrend
            elif pd_eurusd["high_y"].iloc[i] < pd_eurusd["high_y"].iloc[i_start] and pd_eurusd["low_y"].iloc[i] < pd_eurusd["low_y"].iloc[i_start]:
                trend = -1
                SAR = np.append(SAR, pd_eurusd["high_y"].iloc[i_start])
                ext_point = pd_eurusd["high_y"].iloc[i_start]
                i_start = i
                prev_pos = i
                break
            # no trend
            else:
                trend = 0
                SAR = np.append(SAR, np.nan)
                i_start = i
    # else if not good time => nan
    else:
        SAR = np.append(SAR, np.nan)


for i in range(i_start+1, len(pd_eurusd)):
    # check change of trend
    #   => if high or low above or below SAR -> change trend and new SAR == highest high or lowest low

    # else calculate SAR
    if pd_eurusd["time"].iloc[i] % 1800 == 1500:
        # change of trend
        if trend == 1 and pd_eurusd["low_y"].iloc[i] <= SAR[prev_pos]:
            trend = -1
            SAR = np.append(SAR, ext_point)
            ext_point = pd_eurusd["low_y"].iloc[i]
            acc = 0.02
            prev_pos = i
            continue
        elif trend == -1 and pd_eurusd["high_y"].iloc[i] >= SAR[prev_pos]:
            trend = 1
            SAR = np.append(SAR, ext_point)
            ext_point = pd_eurusd["high_y"].iloc[i]
            acc = 0.02
            prev_pos = i
            continue

        # no change of trend => calc SAR
        if trend == 1:
            new_SAR = SAR[prev_pos] + acc * (ext_point - SAR[prev_pos])
            SAR = np.append(SAR, new_SAR)
            if pd_eurusd["high_y"].iloc[i] > ext_point:
                ext_point = pd_eurusd["high_y"].iloc[i]
                if acc < maximum:
                    acc += 0.02
            prev_pos = i
        elif trend == -1:
            new_SAR = SAR[prev_pos] - acc * (SAR[prev_pos] - ext_point)
            SAR = np.append(SAR, new_SAR)
            if pd_eurusd["low_y"].iloc[i] < ext_point:
                ext_point = pd_eurusd["low_y"].iloc[i]
                if acc < maximum:
                    acc += 0.02
            prev_pos = i
    
    else:
        # # change of trend
        
        # no change of trend => calc SAR
        if trend == 1:
            new_SAR = SAR[prev_pos] + acc * (ext_point - SAR[prev_pos])
            SAR = np.append(SAR, new_SAR)
            # if pd_eurusd["high_y"].iloc[i] > ext_point:
            #     ext_point = pd_eurusd["high_y"].iloc[i]
            #     # if acc < maximum:
            #     #     acc += 0.02
        elif trend == -1:
            new_SAR = SAR[prev_pos] - acc * (SAR[prev_pos] - ext_point)
            SAR = np.append(SAR, new_SAR)
            # if pd_eurusd["low_y"].iloc[i] < ext_point:
            #     ext_point = pd_eurusd["low_y"].iloc[i]
            #     # if acc < maximum:
            #     #     acc += 0.02



pd_eurusd["SAR_M30"] = SAR

In [18]:
# Aroon Oscillator * timeperiod = 14

timeperiod = 14
count = 0

highTemp = np.array([])
lowTemp = np.array([])
arrOsc = np.array([])

for i in range(0, len(pd_eurusd)):
    if count < timeperiod-1:
        if pd_eurusd["time"].iloc[i] % 1800 == 1500:
            count += 1
            highTemp = np.append(highTemp, pd_eurusd["high_y"].iloc[i])
            lowTemp = np.append(lowTemp, pd_eurusd["low_y"].iloc[i])
            arrOsc = np.append(arrOsc, np.nan)
        else:
            arrOsc = np.append(arrOsc, np.nan)
    else:
        if pd_eurusd["time"].iloc[i] % 1800 == 1500:
            highTemp = np.append(highTemp, pd_eurusd["high_y"].iloc[i])
            lowTemp = np.append(lowTemp, pd_eurusd["low_y"].iloc[i])
            arrUp = (np.argmax(highTemp) / timeperiod) * 100
            arrDown = (np.argmin(lowTemp) / timeperiod) * 100
            calc = arrUp - arrDown
            arrOsc = np.append(arrOsc, calc)
            highTemp = np.delete(highTemp, 0)
            lowTemp = np.delete(lowTemp, 0)
        else:
            h_temp = np.append(highTemp, pd_eurusd["high_y"].iloc[i])
            l_temp = np.append(lowTemp, pd_eurusd["low_y"].iloc[i])
            arrUp = (np.argmax(h_temp) / timeperiod) * 100
            arrDown = (np.argmin(l_temp) / timeperiod) * 100
            calc = arrUp - arrDown
            arrOsc = np.append(arrOsc, calc)

pd_eurusd["Aroon_M30"] = arrOsc

In [19]:
# Balance Of Power

bop = np.array([])

for i in range(0, len(pd_eurusd)):
    calc = (pd_eurusd["close_y"].iloc[i] - pd_eurusd["open_y"]).iloc[i] / (pd_eurusd["high_y"].iloc[i] - pd_eurusd["low_y"].iloc[i])
    bop = np.append(bop, calc)

pd_eurusd["BOP_M30"] = bop

In [20]:
# CCI * timeperiod = 14

timeperiod = 14
count = 0

typ_price = np.array([])
cci = np.array([])

for i in range(0, len(pd_eurusd)):
    if count < timeperiod-1:
        if pd_eurusd["time"].iloc[i] % 1800 == 1500:
            count += 1
            calc_TP = (pd_eurusd["high_y"].iloc[i] + pd_eurusd["low_y"].iloc[i] + pd_eurusd["close_y"].iloc[i]) / 3
            typ_price = np.append(typ_price, calc_TP)
            cci = np.append(cci, np.nan)
        else:
            cci = np.append(cci, np.nan)
    else:
        if pd_eurusd["time"].iloc[i] % 1800 == 1500:
            calc_TP = (pd_eurusd["high_y"].iloc[i] + pd_eurusd["low_y"].iloc[i] + pd_eurusd["close_y"].iloc[i]) / 3
            typ_price = np.append(typ_price, calc_TP)
            sma_TP = talib.SMA(typ_price, timeperiod = timeperiod)
            mean_dev = np.mean(np.absolute(typ_price - sma_TP[-1]))
            calc = (typ_price[-1] - sma_TP[-1]) / (0.015 * mean_dev)
            cci = np.append(cci, calc)
            typ_price = np.delete(typ_price, 0)
        else:
            calc_TP = (pd_eurusd["high_y"].iloc[i] + pd_eurusd["low_y"].iloc[i] + pd_eurusd["close_y"].iloc[i]) / 3
            typ_p = np.append(typ_price, calc_TP)
            sma_TP = talib.SMA(typ_p, timeperiod = timeperiod)
            mean_dev = np.mean(np.absolute(typ_p - sma_TP[-1]))
            calc = (typ_p[-1] - sma_TP[-1]) / (0.015 * mean_dev)
            # print(mean_dev)
            cci = np.append(cci, calc)

pd_eurusd["CCI_M30"] = cci

In [21]:
# pd_eurusd.describe()
test = pd.DataFrame(pd_eurusd, columns=["time", "open_y", "close_y", "high_y", "low_y", "CCI_M30"])
# print(test[test["time"] % 1800 == 1500][-50:])
print(test[-50:])


            time   open_y  close_y   high_y    low_y     CCI_M30
9950  1613750400  1.21307  1.21267  1.21325  1.21261  -24.543896
9951  1613750700  1.21307  1.21267  1.21325  1.21233  -32.771997
9952  1613751000  1.21307  1.21239  1.21325  1.21233  -40.794242
9953  1613751300  1.21307  1.21239  1.21325  1.21227  -42.487252
9954  1613751600  1.21307  1.21233  1.21325  1.21227  -44.171234
9955  1613751900  1.21307  1.21192  1.21325  1.21188  -65.629855
9956  1613752200  1.21190  1.21224  1.21228  1.21175 -119.863106
9957  1613752500  1.21190  1.21193  1.21228  1.21175 -128.640630
9958  1613752800  1.21190  1.21222  1.21225  1.21175 -121.297894
9959  1613753100  1.21190  1.21243  1.21254  1.21175 -106.606819
9960  1613753400  1.21190  1.21252  1.21254  1.21175 -103.879097
9961  1613753700  1.21190  1.21261  1.21262  1.21175  -98.654529
9962  1613754000  1.21261  1.21202  1.21265  1.21202 -135.090268
9963  1613754300  1.21261  1.21201  1.21265  1.21190 -139.417227
9964  1613754600  1.21261

In [22]:
# A1 = 0
# A2 = 1

# SI(A1=0, "zero", "pouet")

# if A1 == 0:
#     print("zero")
# else:
#     print("pouet")